# Recommendation based on EDA and visualization of retail store data

[This is my first notebook. If you like it please do UPVOTE]

# Business understanding
The BigMart food and grocery retail has been operating since 1985. Currently it is facing problem of low revenue. It intends to find underlining problem resulting to it and which products to focus on to increase store revenues. 

# Data understanding 
2013 sales data has been collected across its stores in different cities to analyse the problem

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#data loading and view of data
import pandas as pd
file = '/kaggle/input/big-mart-sales/train_v9rqX0R.csv'
df = pd.read_csv(file)
df.head()

In [ ]:
print(df.shape) #dimension of data: 8523 rows, 12 columns
print(df.dtypes) #datatypes: measures(4) and categorical variables
print(df["Item_Type"].unique()) #to get unique entries
print(len(df["Item_Type"].unique())) #16 categories of products available
print(len(df["Item_Identifier"].unique())) #1559 unique products types
print(len(df["Outlet_Identifier"].unique())) #10 outlets
print(df["Outlet_Establishment_Year"].unique()) # 1985-2009
print(len(df["Outlet_Establishment_Year"].unique()))

In [ ]:
#to find null values in data
df.isna().sum() #found in size of store no: 10,17,45 and item weight in store no:19,27

# Data Preparation and Cleaning

In [ ]:
df['Item_Weight'].fillna(min(df["Item_Weight"]), inplace = True)  
df['Outlet_Size'].fillna("NA", inplace = True)  
df['Item_Fat_Content'].replace(to_replace="LF", value="Low Fat",inplace = True)
df['Item_Fat_Content'].replace(to_replace="low fat", value="Low Fat",inplace = True)
df['Item_Fat_Content'].replace(to_replace="reg", value="Regular",inplace = True)

#print(df.iloc[7])
#print(df.iloc[3])
#print(df.iloc[65])
#print(df.iloc[81])

In [ ]:
df.dropna(inplace=True) #deletes row which has atleast 1 na
print(df.shape)

In [ ]:
#checking null presence after making changes
df.isna().sum()

# Visualization with Analysis

In [ ]:
#pip install seaborn
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#types of store
#sns.catplot(y=df["Outlet_Identifier"],x=df["Outlet_Type"], palette="pastel", data=df)
sns.countplot(y=df["Outlet_Type"], data=df)

In [ ]:
sns.countplot(y=df["Outlet_Identifier"], data=df)

### Shows 10 store outlets are distributed among 4 types of store. Maximum number of stores are of  Supermarket-1 type

In [ ]:
#distribution of store across location
#sns.stripplot(y=df["Outlet_Identifier"],x=df["Outlet_Location_Type"], palette="pastel", data=df)
sns.countplot(y=df["Outlet_Location_Type"], data=df)

### All stores in location Tier-2 have only Supermarket-1 type of stores. Location Tier-3 has mix of all 4 types of stores. Location Tier-1 has only 1 grocery store and rest 2 are of Supermarket-1 type.

In [ ]:
#Store types,size,location,establishment year, outlet no, sales

year_sale_dx = df.groupby(by=['Outlet_Type', 'Outlet_Identifier','Outlet_Establishment_Year','Outlet_Location_Type','Outlet_Size'])['Item_Outlet_Sales'].sum().reset_index()
#year_sale = year_sale_dx.groupby(by=['Item_Type'])['Item_Outlet_Sales'].transform(min) == year_sale_dx['Item_Outlet_Sales']
#year_sale_min = year_sale_dx[year_sale].reset_index(drop=True)
year_sale_dx.sort_values(by='Item_Outlet_Sales')

Store of Grocery type(that is store outlet-no 10 and 19) has very low revenue and store outlet no-27 of type SuperMarket Type-3 has highest revenue of all items sold. Store type wise ascending order of store revenue are: grocery,supermarket type-2(store no:18),supermarket type- 1(store no:45,46,13,17,49,35), supermarket type-3.

### Size analysis of store types:
Size of Grocery type store no:10 and Supermarket Type-1 store no:17,45 is not available in data. 
Grocery Store outlet no:19 is of small size of store. Supermarket Type-2,3 store is of medium size. Supermarket Type-1 have all 3 types of size. 
#### Thus in this data revenue is independent of store size


### Location effect on revenue that is sales based on customer buying: 
Presence of store outlet no-27 in Tier-3 is affecting revenue of other stores in that location(visbible from above plots) so we less revenue in stores numbers: 13(supermarket type- 1),10(Grocery type),18(supermarket type-2). It may be popularity of store outlet no-27 is resulting less visit thus less sales to other types of store in that location. Store 19 is located in Tier-1 where other stores are of type Supermarket-1 which are having better popularity than grocery type stores. Tier-2 location generates better revenue for supermarket type- 1 than Tier-1.


### Establishment year of store types:
Grocery store outlet:19 and Supermarket Type-3 outlet are oldest stores. After 13 years new grocery store was established in Tier-3 which is same location as Supermarket Type-3, maybe because Store no:27 was having better revenue in that location.

After 2 years, Supermarket Type-1 outlet was established again(maybe because it had better revenue than grocery type store)  in location Tier-3. After 10 years of wait company decided to establish another store of Supermarket Type-1 in location Tier-1 same where 1st grocery type store was established (maybe because it didnt have highest among all stores revenue in previous location). After 2 years another store of Supermarket Type-1,  was established in same location Tier-1 may be to boost sales in the region.

After 3 years company thought of exploring new location, so set up Supermarket Type-1 in Tier-2. After 2 years another store of same type,  was established in same location may be to boost sales in the region. Maybe as this store had nearly good revenue as store no:27, in the same location, another store of Supermarket Type-1 was added to same location Tier-2. This time may be revenue decreased and company stopped establishing more stores till 2013.
#### We can observe a pattern that the retail chain had habit of establishing new stores very often like within 1-2 years. There maybe possibility that whenever they found revenue of new store decreasing than previous they changed location for new store to be established.


### Hypotheses-1: More revenue in store outlets may be due to more product line items available (better stocks) under each item type/category.
### Hypotheses-2: More revenue in stores may be due to more quantity sales of products 
### Hypotheses-3: % of product visibility may be due to size of store or item weight.

In [ ]:
#Analysis of sales of item types in Grocery store
year_sale_dx = df.groupby(by=['Outlet_Type', 'Item_Type'])['Item_Outlet_Sales'].sum().reset_index()
year_sale = year_sale_dx.groupby(by=['Item_Type'])['Item_Outlet_Sales'].transform(min) == year_sale_dx['Item_Outlet_Sales']
year_sale_min = year_sale_dx[year_sale].reset_index(drop=True)
year_sale_min.sort_values(by='Item_Outlet_Sales')

Shows Grocery stores sell all 16 types of items. Details of products falling under "other" categories is not present in given data.
### Seafood, Starchy Foods, Hard Drinks, "others" and Breakfast contribute to least revenue.

### Hypotheses-4: Reason of least revenue in grocery store is due to low mean price under each category of products OR
### Less quantity sales of products or due to very less number of product lines(Hypotheses-1) under that category compared to other stores

In [ ]:
#count of product items of each category in Grocery store
year_sale_dx = df.groupby(by=['Outlet_Type','Item_Type'])['Item_Identifier'].count().reset_index()
year_sale_dx.head(17)

In [ ]:
#count of product items of each category in Outlet 27 (comparison with highest revenue store)
year_sale_dx = df.groupby(by=['Outlet_Type','Item_Type'])['Item_Identifier'].count().reset_index()
year_sale_dx.tail(17)

### This proves Hypotheses-1 Wrong as we see grocery store is keeping much more number of product items under each item type. So it is not a reason of low revenue in grocery stores. 

### It may be stacking of large number of item in Small Size grocery stores compared to medium size store of outlet-27 may result in low Visibility of items(Hypotheses-3) hence leading to less quantity of sales

In [ ]:
#Visibility of item types in Grocery store
year_sale_dx = df.groupby(by=['Outlet_Type','Outlet_Identifier','Item_Type'])['Item_Visibility'].mean().reset_index()
year_sale_dx.head(33)

In [ ]:
#Visibility of item types in Grocery store
year_sale_dx = df.groupby(by=['Outlet_Type','Item_Type'])['Item_Visibility'].mean().reset_index()
year_sale_dx.tail(17)

### This proves Hypotheses-3 Wrong as it shows that visibility of products in each category is not affected by store size neither it is affecting sales of products under each item type as in Grocery stores each item category has better visibity than outlet-27 

In [ ]:
#Correlation analysis between variables
df['Item_Visibility'].replace(to_replace="NA", value=min(df["Item_Visibility"]),inplace = True)
corr = df.corr()
ax = sns.heatmap(corr, vmin=-1, vmax=1, center=0)

### Shows moderate positive coorelation between MRP and sales. Rest variables are not correlated to each other.
### Thus supports Hypotheses-4.

In [ ]:
#Min mean Price Comparison between stores to study affect on quantity sales
year_sale_dx = df.groupby(by=['Outlet_Type', 'Item_Type'])['Item_MRP'].mean().reset_index()
year_sale = year_sale_dx.groupby(by=['Item_Type'])['Item_MRP'].transform(min) == year_sale_dx['Item_MRP']
year_sale_min = year_sale_dx[year_sale].reset_index(drop=True)
year_sale_min.sort_values(by='Item_MRP')

Supermarket Type-3 keeps min mean price of items under categories: "others",soft drinks,seafood,starchy food,snacks,household. Supermarket Type-2 keeps min mean price of items under categories: health and hygiene, breakfast, breads,fruits and vegetables,dairy. 
Supermarket Type-1 keeps min mean price of items only under category:baking goods.

Grocery Type: keeps min mean price of items under category: hard drinks, frozen food, meat, canned

### This also shows reason of less revenue generated from Hard Drinks in Grocery type stores

### But items categories Seafood, Starchy Foods, Breakfast does not falls under minimum mean price category offered across stores. So it is left to investigate.

In [ ]:
#mean price of products under each item category in Grocery store
year_sale_dx = df.groupby(by=['Outlet_Type','Item_Type'])['Item_MRP'].mean().reset_index()
year_sale_dx.head(17)

In [ ]:
#mean price of products under each item category in Grocery store
year_sale_dx = df.groupby(by=['Outlet_Type','Item_Type'])['Item_MRP'].mean().reset_index()
year_sale_dx.tail(17)

Shows supermarket type-3 keeps only slight increase in price than grocery type stores. Only for item categories: breads,health and hygiene, household, others, seafood,snack, soft drink price is slightly higher in Grocery stores. 

### This shows less quantity sales of item is the main cause of low revenue in grocery stores which proves Hypotheses-2. But this is not entirely due to difference in mean price set by different stores.



## Patterns of sales of products in BigMart

In [ ]:
#analysis of count and sales under each product category across stores
fig, axes = plt.subplots(3, 1, figsize=(10, 10))
sns.countplot(ax=axes[0],y=df["Item_Type"],palette="pastel", data=df)
sns.barplot(ax=axes[1],y=df["Item_Type"],x=df["Item_Outlet_Sales"], palette="pastel", data=df)
sns.barplot(ax=axes[2],y=df["Item_Type"],x=df["Item_MRP"], palette="pastel", data=df)

### From plot 2 and 3 it is visible that mean Revenue across product categories across stores is proportional to mean MRP and mean Revenue across product categories is not having major difference.

### Item Type wise analysis in Big Mart:
There is large variation of mean price across stores in items under Seafood, Breakfast and Starchy food categories and least variation in Fruits and Vegetables, Snacks, Households. May either speak on buying capacity of customers based on store location or quality of product in the store outlet.

Categories "others", Breakfast, Baking Goods, Soft Drinks records lowest Revenue in some stores as their mean MRP is also lowest except for Breakfast. Mean revenue is high for: Seafood, Starchy food, Fruits and Vegetables, Snacks, Households, Dairy, Canned as their mean price are also high. Mean revenue is least for: "others",Baking Goods, Soft Drinks, Health and Hygiene


Breakfast category must have least quantity sales as number of product lines are also very less for it. [if drink are offered at comparable price, more people will prefer Hard Drinks]. Items under "others" is unknown so no decision can be taken.

Comparison of number of product lines under each item type with Revenue:
categories under Seafood, Breakfast, starchy food, "others", hard drinks have least types of products under it and categories: fruits, snacks have highest types of products types under it. This shows Seafood, starchy food, "others", hard drinks have very high quantity of sales talks about either buying pattern of customers.

In [ ]:
#sales of fat content products
fig, axes = plt.subplots(3, 1, figsize=(10, 10))
sns.countplot(ax=axes[0],x=df["Item_Fat_Content"],palette="pastel", data=df)
sns.barplot(ax=axes[1],x=df["Item_Fat_Content"],y=df["Item_Outlet_Sales"], palette="pastel", data=df)
sns.barplot(ax=axes[2],x=df["Item_Fat_Content"],y=df["Item_MRP"], palette="pastel", data=df)

Shows BigMart keeps higher quantity of Low Fat types product but keeps similar price for both Regular and Low fat types of products.
### So there is not much difference in total revenue generated between them which shows low quantity sales of Low Fat types product

# Recommendations

### I) Location+ store type: 
If BigMart decides on establish new store of type: Supermarket-1 it should choose location Tier-2 [as Supermarket-1 type of outlets located there was generating more revenue than other locations] or explore a new location.
    
It can try opening new store of type Supermarket-3 as only one outlet is established till date and it generated maximum revenue among all stores. So it should be located either in a new location or in Tier-1 to improve revenue generated in that location and not undermine good sales at other location.

### II) Which revenue generating product category to stock more or introduce more line items under it
Mean revenue is high for item types Seafood, Starchy food inspite of very low count of product lines under them which indicate more quantity of sales thus shows higher demand for these items among customers across all locations. So they should be kept large in stock and sold at high prices especially in low revenue generating stores.

More number of products to be introduced under Breakfast category to boost its mean revenue. As we see quantity of sales in this category is low due to less number of variety of items present under this category. Also for some stores it recorded highest revenue due to high price which shows demand exist for this category but due to less variety there is less quantity sales in stores.

### III) Which product category to discard to increase visibity of other products and save more space for stocking of better revenue generating products

Softdrinks must be replaced with Hard Drinks. As we observe higher demand of Hard Drinks in stores as it generates more revenue when price of both kept similar and currently Softdrinks falls under lowest mean revenue generating category.

Low Fat types products must be replaced with Regular types products as not much demand is seen in the former when prices of both types were kept same as can also be seen in greater revenue in Snacks category of food items.

### IV)Whether change in price required

Products under "others" category to be investigated as it generates lowest mean revenue. If any item under it is found to fall under customer preference, its price to be increased to check whether it increases revenue as well. If not items under it can be replaced by new product lines.

Baking Goods and Health and Hygiene fall under low revenue generating category as their mean selling price is also low. 
So price increase in these categories can be tried to increase revenue.

### V)Choosing from current scenario of Top and Bottom 10 product item names
Product item FDR45(item type: Snacks food) generate highest mean revenue and FDY55(item type: Fruits and vegetables) generated highest sum of revenue in Big Mart sales irespective of any groups/stores. FD539(item type: Dairy) generated high revenue alone from store outlet no-13(location: Tier-3, supermarket-1 type). More number of these top 10 products belong to categories: Fruits and Vegetables(3 items)and  Dairy and Frozen Foods (2 items). 

Most of the items generating highest revenue from single store comes from store outlet-27. Highest revenue generating product items are NCE42(item type: Household). NCM05(item type: health and hygiene) generated high revenue by single sales from store outlet-46(location: Tier-1, supermarket-1 type). More number of these top 10 products coming from single store sales belong to categories: Household(2 items), Fruits and Vegetables(2 items)and  Dairy(2 items).

Product item NCR42(item type: household) generate lowest mean revenue alone from single store Outlet-10(Grocery store in Tier-3 location). and FDQ60(item type: baking goods) generated least sum of revenue in Big Mart sales irespective of any groups/stores. More number of these bottomost 10 products belong to categories: Household,Health and Hygiene, dairy(2 items of each category).
Most of the items generating lowest revenue from single store comes from store outlet-10 and 19 grocery stores. Lowest revenue generating product items are DRK12(item type: Soft Drinks) from store outlet-10 and FDU21(item type: Snack foods) from store outlet-19. More number of these bottomost 10 products coming from single store sales belong to categories:Frozen foods(2 items).

#### So these product items(NCR42,FDQ60,DRK12,FDU21) should be discarded as already there is better revenue generating products in Snack,Frozen Food,household categories in other stores.

### VI)Few other recommendations:
BigMart should find new ways to increase quantity sales especially in grocery store type apart from increasing price and exploring location. It may spend on advertisement, infrastructure, providing better employee service, easy of shopping to attract people to visit its grocery stores(From proof of hypotheses-2)

It should ensure freshness of perishable items like Seafood (falling under high revenue generating items) and also other perishable food items particularly in Grocery store.

Price of items in Grocery stores should be increased to increase revenue only if the new location has customers with higher buying capability or by advertising and ensuring premium quality of product. 

In [ ]:
#products having highest revenue across all stores
cg=df.groupby('Item_Identifier')
z=cg['Item_Outlet_Sales'].agg(['mean','sum']).sort_values(by='mean', ascending=False)
z.head(10)

#### Shows top 10 product items having highest mean revenue in Big Mart across stores. Product item FDR45 had 

In [ ]:
#under which item type does these products fall under: 

type=['FDR45','NCL42','FDU55','FDZ50','DRK23','FDF39','FDD44','FDT16','FDY55','FDG17']
filt=df['Item_Identifier'].isin(type)
df.loc[filt,['Item_Type','Item_Identifier']]

#### They fall under item type: Snack foods, Household, Fruits and Vegetables(3 items), Dairy(2 items), Hard Drinks, Frozen Foods (2 items)

In [ ]:
#Top 10 product items showing in which store they have highest revenue

year_sale_dx = df.groupby(by=['Outlet_Identifier','Item_Type','Item_Identifier'])['Item_Outlet_Sales'].sum().reset_index()
z=year_sale_dx.sort_values(by='Item_Outlet_Sales',ascending=False)
z.head(10)

#### top 10 product items having highest sum of revenue and in which store outlet.

In [ ]:
#Bottomost 10 products having lowest revenue across all stores
cg=df.groupby('Item_Identifier')
z=cg['Item_Outlet_Sales'].agg(['mean','sum']).sort_values(by='mean')
z.head(10)

#### Bottomost 10 product items having lowest sum of revenue in BigMart. So they should not be stocked in stores.

In [ ]:
#under which item type does these products fall under: 

type=['NCR42','FDQ60','FDQ04','FDX10','NCN29','NCL41','NCO06','FDF38','FDT02','FDX38']
filt=df['Item_Identifier'].isin(type)
df.loc[filt,['Item_Type','Item_Identifier']]

#### They fall under item type: Household(2 items),Baking goods, Frozen foods, Snack Foods, Health and Hygiene(2 items), canned, dairy(2 items)

In [ ]:
#Bottomost 10 product items showing in which store they have lowest revenue

year_sale_dx = df.groupby(by=['Outlet_Identifier','Item_Type','Item_Identifier'])['Item_Outlet_Sales'].sum().reset_index()
z=year_sale_dx.sort_values(by='Item_Outlet_Sales')
z.head(10)

In [ ]:
 #If you like it please do click UPVOTE button